In [82]:
import pandas as pd
import ast
import collections

In [3]:
tweets = pd.read_csv('tweets_csv.csv')
tweets

,Unnamed: 0.1,Unnamed: 0,date,rawContent,user,lang,place,hashtags,likeCount,retweetCount
0,0,0,2018-08-01 07:42:50,Opened the shutters of #Malampuzha dam #Palakk...,"{'username': 'AdvSudhinSuresh', 'id': 77953040...",en,"{'id': '09294260a5199155', 'fullName': 'Ulliye...","['Malampuzha', 'Palakkad', 'KeralaFlood']",0,0
1,1,1,2018-08-09 05:02:39,It's rising. Stay safe. #Keralarains#kalady ht...,"{'username': 'tharunova', 'id': 80231605, 'dis...",en,"{'id': '7a01d1cbca786e81', 'fullName': 'Kanaya...",NaN,1,0
2,2,2,2018-08-09 05:42:24,More pics from the scene. #kalady#keralaflood ...,"{'username': 'tharunova', 'id': 80231605, 'dis...",en,"{'id': '7a01d1cbca786e81', 'fullName': 'Kanaya...",NaN,0,0
3,3,3,2018-08-09 07:07:00,#idukki dams third shutter opened for first ti...,"{'username': 'shanksnews', 'id': 4656066013, '...",en,"{'id': '5d01806d080d1e50', 'fullName': 'Kerala...","['idukki', 'Kerala', 'periyar', 'aluva', 'Kera...",0,0
4,4,4,2018-08-09 07:26:31,Idukki dam opened after 26 years!\n#IDUKKI \n#...,"{'username': 'ChandyJongee', 'id': 10159452395...",en,"{'id': '24dab5a953e050c5', 'fullName': 'Trivan...","['IDUKKI', 'keralarains']",0,0
...,...,...,...,...,...,...,...,...,...,...
2650,2650,2650,2018-08-30 06:51:13,2 croces from Shriram transport Finance compan...,"{'username': 'KalaiselvanJ', 'id': 357352106, ...",en,"{'id': '5a75c024c6ae075b', 'fullName': 'Uthira...",['KeralaFloods'],1,0
2651,2651,2651,2018-08-30 10:48:18,After a successful show in coimbatore we are p...,"{'username': 'theatrekaran', 'id': 83941462862...",en,"{'id': '4ad0fa6d89777d40', 'fullName': 'Egmore...","['theatrekaran', 'Stories', 'KeralaFloods', 'h...",0,1
2652,2652,2652,2018-08-30 12:03:01,@AAPExpress @ArvindKejriwal @SatyendarJain Tha...,"{'username': 'albypanakalam', 'id': 75216041, ...",en,"{'id': '5f55bb82cf16ac81', 'fullName': 'Bengal...","['StandwithKerala', 'KeralaFloods', 'KeralaFlo...",0,0
2653,2653,2653,2018-08-30 16:08:51,"In Kerala, there’s no reason to damn the dams ...","{'username': 'Tom_Babu', 'id': 635996364, 'dis...",en,"{'id': '24dab5a953e050c5', 'fullName': 'Trivan...",['KeralaFloods'],0,1


In [4]:
print(tweets.dtypes)

Unnamed: 0.1     int64
Unnamed: 0       int64
date            object
rawContent      object
user            object
lang            object
place           object
hashtags        object
likeCount        int64
retweetCount     int64
dtype: object


In [8]:
df = tweets.drop(columns=['Unnamed: 0.1','Unnamed: 0','likeCount','retweetCount','user','lang'])
df.head()

,date,rawContent,place,hashtags
0,2018-08-01 07:42:50,Opened the shutters of #Malampuzha dam #Palakk...,"{'id': '09294260a5199155', 'fullName': 'Ulliye...","['Malampuzha', 'Palakkad', 'KeralaFlood']"
1,2018-08-09 05:02:39,It's rising. Stay safe. #Keralarains#kalady ht...,"{'id': '7a01d1cbca786e81', 'fullName': 'Kanaya...",NaN
2,2018-08-09 05:42:24,More pics from the scene. #kalady#keralaflood ...,"{'id': '7a01d1cbca786e81', 'fullName': 'Kanaya...",NaN
3,2018-08-09 07:07:00,#idukki dams third shutter opened for first ti...,"{'id': '5d01806d080d1e50', 'fullName': 'Kerala...","['idukki', 'Kerala', 'periyar', 'aluva', 'Kera..."
4,2018-08-09 07:26:31,Idukki dam opened after 26 years!\n#IDUKKI \n#...,"{'id': '24dab5a953e050c5', 'fullName': 'Trivan...","['IDUKKI', 'keralarains']"


In [11]:
df.describe()

,date,rawContent,place,hashtags
count,2655,2655,2557,2627
unique,2651,2650,156,1568
top,2018-08-18 06:52:11,@KSAMOFA @KSAmofaEN @KingSalman @KingSalmanEn ...,"{'id': '5d01806d080d1e50', 'fullName': 'Kerala...",['KeralaFloods']
freq,2,3,432,470


In [12]:
districts = ['Alappuzha', 'Ernakulam', 'Kozhikode', 'Palakkad', 'Kollam', 'Kannur', 'Kasaragod', 'Idukki', 'Kottayam', 'Thrissur', 'Pathanamthitta', 'Malappuram', 'Wayanad', 'Thiruvananthapuram']

In [14]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2530 entries, 0 to 2654
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        2530 non-null   object
 1   rawContent  2530 non-null   object
 2   place       2530 non-null   object
 3   hashtags    2530 non-null   object
dtypes: object(4)
memory usage: 98.8+ KB


In [92]:
locations = df['place'].to_list()
print(type(locations[1]))
locations[2]


<class 'str'>


"{'id': '24dab5a953e050c5', 'fullName': 'Trivandrum, India', 'name': 'Trivandrum', 'type': 'city', 'country': 'India', 'countryCode': 'IN'}"

In [80]:
place_list = []
for i in range(2530):
    test_string = locations[i]
    res = ast.literal_eval(test_string)
    place_list.append(res['name'])
place_list

['Ulliyeri',
 'Kerala',
 'Trivandrum',
 'Bengaluru South',
 'Saidapet',
 'Mylapore Tiruvallike',
 'Munnar',
 'Munnar',
 'Egmore Nungambakkam',
 'Munnar',
 'Trivandrum',
 'Kanayannur',
 'Munnar',
 'Kanayannur',
 'Tamil Nadu',
 'Bengaluru',
 'Tamil Nadu',
 'Egmore Nungambakkam',
 'Trivandrum',
 'Trivandrum',
 'Kalpetta',
 'Bengaluru South',
 'Chennai',
 'Trissur',
 'Egmore Nungambakkam',
 'Bengaluru South',
 'Saidapet',
 'Kerala',
 'Ambalappuzha-Alappuzha-Komalapuram-Pathirapally',
 'Cherthala',
 'Kerala',
 'Kerala',
 'Kerala',
 'Egmore Nungambakkam',
 'Egmore Nungambakkam',
 'Trivandrum',
 'Saidapet',
 'Bengaluru',
 'Kerala',
 'Perungudi',
 'Bengaluru',
 'Kerala',
 'Anekal Bangaluru',
 'Malappuram (Ernad)',
 'Ambalappuzha-Alappuzha-Komalapuram-Pathirapally',
 'Sholinganallur',
 'Ambalappuzha-Alappuzha-Komalapuram-Pathirapally',
 'Kerala',
 'Trivandrum',
 'Trivandrum',
 'Trivandrum',
 'Cochin',
 'Kerala',
 'Kollam',
 'Cochin',
 'Egmore Nungambakkam',
 'Cochin',
 'Kanayannur',
 'Trivandru

In [86]:
place_df = pd.DataFrame(place_list)
place_df

,0
0,Ulliyeri
1,Kerala
2,Trivandrum
3,Bengaluru South
4,Saidapet
...,...
2525,Uthiramerur
2526,Egmore Nungambakkam
2527,Bengaluru South
2528,Trivandrum


In [88]:
place_df.describe()

,0
count,2530
unique,155
top,Kerala
freq,431
